# Music Generation

##### Import necesary packages

In [5]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.


##### Load the pre-processed notes saved earlier

In [0]:
notes = []

with open ("notes", "rb") as file:
    notes = pickle.load(file)

In [8]:
print ("Total number of notes = " + str(len(notes)))

# Get total numeber of classes for the model
vocab_len = len(set(notes))
print("Total number of classes (i.e. numeber of unique notes) = " + str(vocab_len))

Total number of notes = 60498
Total number of classes (i.e. numeber of unique notes) = 359


In [9]:
print(notes)

['F3', 'F3', 'B-4', 'F3', 'G4', 'G#4', 'F4', 'F3', 'G4', 'G3', 'E-4', 'G#3', 'F3', 'F5', 'G5', 'G#5', 'F3', 'B-5', 'G5', 'G#5', 'F3', 'B-5', 'C6', 'B-5', 'F3', 'E-6', 'C#4', 'F6', 'C4', 'F3', 'F3', 'B-4', 'F3', 'G4', 'G#4', 'F4', 'F3', 'G4', 'G3', 'E-4', 'G#3', 'F3', 'F5', 'G5', 'G#5', 'F3', 'B-5', 'G5', 'G#5', 'F3', 'B-5', 'C6', 'B-5', 'F3', 'E-6', 'E-3', 'F6', 'E3', 'F2', 'F2', 'B-4', 'F2', 'G4', 'G#4', 'B-2', 'F4', 'G4', 'C3', 'E-4', 'F2', 'F5', 'G5', 'G#5', 'F2', 'B-5', 'G5', 'G#5', 'F2', 'B-5', 'C6', 'B-5', 'F2', 'E-6', 'C#3', 'F6', 'C3', 'F2', 'F2', 'B-4', 'F2', 'G4', 'G#4', 'B-2', 'F4', 'G4', 'C3', 'E-4', 'F2', 'F5', 'G5', 'G#5', 'F2', 'B-5', 'G5', 'G#5', 'F2', 'B-5', 'C6', 'B-5', 'F2', 'E-6', 'E-2', 'F6', 'E2', '0+5', 'F2', 'F2', 'F2', 'B-2', 'C3', '8+0', 'F2', 'F2', '7+10', 'F2', 'F2', 'C#3', 'C3', '0+3', 'F2', 'F2', 'F2', 'B-2', 'C3', '10+2', 'F2', 'F2', 'F2', 'F2', 'E-2', 'E2', '8+0', 'F2', 'F2', 'F2', 'B-2', 'C3', 'F2', 'F2', 'F2', 'F2', 'C#3', 'C3', '5', 'F2', '5', 'F2', '

#### Prepare Sequential data for LSTM

In [0]:
# Decide on the maximum number of elements the LSTM model should consider
sequence_length = 100

# Store the desired classes
pitch_names = sorted(set(notes))

In [0]:
# Create a mapping from notes to integer
element_to_int = dict( (elem, num) for num, elem in enumerate(pitch_names))

In [12]:
print(element_to_int)

{'0': 0, '0+1': 1, '0+1+5': 2, '0+1+6': 3, '0+2': 4, '0+2+3+7': 5, '0+2+4+7': 6, '0+2+5': 7, '0+2+6': 8, '0+2+7': 9, '0+3': 10, '0+3+5': 11, '0+3+5+8': 12, '0+3+6': 13, '0+3+6+8': 14, '0+3+6+9': 15, '0+3+7': 16, '0+4': 17, '0+4+5': 18, '0+4+6': 19, '0+4+7': 20, '0+5': 21, '0+5+6': 22, '0+6': 23, '1': 24, '1+2': 25, '1+2+4+6+8+10': 26, '1+2+6': 27, '1+2+6+8': 28, '1+3': 29, '1+3+5': 30, '1+3+5+8': 31, '1+3+6': 32, '1+3+7': 33, '1+3+8': 34, '1+4': 35, '1+4+6': 36, '1+4+6+9': 37, '1+4+7': 38, '1+4+7+10': 39, '1+4+7+9': 40, '1+4+8': 41, '1+5': 42, '1+5+8': 43, '1+5+9': 44, '1+6': 45, '1+7': 46, '10': 47, '10+0': 48, '10+0+2+5': 49, '10+0+3': 50, '10+0+4': 51, '10+0+5': 52, '10+1': 53, '10+1+3': 54, '10+1+3+5+6': 55, '10+1+3+6': 56, '10+1+4': 57, '10+1+4+6': 58, '10+1+5': 59, '10+11': 60, '10+11+3': 61, '10+11+3+5': 62, '10+2': 63, '10+2+3': 64, '10+2+4': 65, '10+2+5': 66, '10+3': 67, '11': 68, '11+0': 69, '11+0+4': 70, '11+0+4+6': 71, '11+0+4+7': 72, '11+0+5': 73, '11+1': 74, '11+1+4': 75,

In [0]:
# Create the input and output for the LSTM model

# Total number of notes is 60498.

# Take first sequence of notes (of length = sequence_length) as first input data point.
# Then ground-truth output for this data point will be the next element in notes (i.e. notes[1 + sequence_length])

# Similarly obtain next data point = notes[2 : 2+sequence_length]
# Expected output for this = notes[2 + sequence_length]

network_input = []
network_output = []

for i in range (len(notes) - sequence_length):
    seq_inp = notes[i : i+sequence_length]
    seq_out = notes[i + sequence_length]

    network_input.append ([element_to_int[ch] for ch in seq_inp])
    network_output.append(element_to_int[seq_out])

In [14]:
print(network_output)

[351, 342, 293, 357, 351, 342, 293, 314, 293, 342, 328, 324, 346, 330, 21, 342, 342, 342, 290, 311, 236, 342, 342, 215, 342, 342, 304, 311, 10, 342, 342, 342, 290, 311, 63, 342, 342, 342, 342, 324, 330, 236, 342, 342, 342, 290, 311, 342, 342, 342, 342, 304, 311, 171, 342, 171, 342, 292, 342, 356, 350, 290, 344, 356, 311, 326, 342, 345, 357, 351, 342, 293, 357, 351, 342, 293, 314, 293, 342, 328, 304, 346, 311, 21, 342, 342, 342, 290, 311, 236, 342, 342, 215, 342, 342, 304, 311, 10, 342, 342, 342, 290, 311, 63, 342, 342, 342, 342, 324, 330, 236, 342, 342, 342, 290, 311, 342, 342, 342, 342, 304, 311, 171, 342, 171, 342, 292, 342, 356, 350, 290, 344, 356, 311, 326, 342, 345, 357, 351, 342, 293, 357, 351, 342, 293, 314, 293, 342, 328, 304, 346, 311, 42, 303, 303, 303, 304, 311, 172, 290, 290, 240, 290, 290, 290, 21, 342, 342, 342, 290, 311, 342, 342, 342, 342, 324, 330, 42, 303, 303, 303, 304, 311, 42, 290, 290, 21, 290, 290, 290, 21, 342, 342, 342, 290, 311, 342, 342, 342, 342, 324, 330, 3

In [15]:
print(network_input[0])

[343, 343, 292, 343, 356, 350, 344, 343, 356, 355, 326, 349, 343, 345, 357, 351, 343, 293, 357, 351, 343, 293, 314, 293, 343, 328, 305, 346, 312, 343, 343, 292, 343, 356, 350, 344, 343, 356, 355, 326, 349, 343, 345, 357, 351, 343, 293, 357, 351, 343, 293, 314, 293, 343, 328, 325, 346, 331, 342, 342, 292, 342, 356, 350, 290, 344, 356, 311, 326, 342, 345, 357, 351, 342, 293, 357, 351, 342, 293, 314, 293, 342, 328, 304, 346, 311, 342, 342, 292, 342, 356, 350, 290, 344, 356, 311, 326, 342, 345, 357]


In [0]:
# Stote number of examples
n_patterns = len(network_input)

In [17]:
print(n_patterns)

60398


In [18]:
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(60398, 100, 1)


In [0]:
# Make all values between 0-1
network_input = network_input/float(vocab_len)

In [20]:
print(network_input)

[[[0.95543175]
  [0.95543175]
  [0.81337047]
  ...
  [0.95264624]
  [0.96100279]
  [0.99442897]]

 [[0.95543175]
  [0.81337047]
  [0.95543175]
  ...
  [0.96100279]
  [0.99442897]
  [0.97771588]]

 [[0.81337047]
  [0.95543175]
  [0.99164345]
  ...
  [0.99442897]
  [0.97771588]
  [0.95264624]]

 ...

 [[0.95543175]
  [0.90529248]
  [0.95264624]
  ...
  [0.87465181]
  [0.8551532 ]
  [0.87465181]]

 [[0.90529248]
  [0.95264624]
  [0.90807799]
  ...
  [0.8551532 ]
  [0.87465181]
  [0.3454039 ]]

 [[0.95264624]
  [0.90807799]
  [0.3454039 ]
  ...
  [0.87465181]
  [0.3454039 ]
  [0.95264624]]]


In [0]:
# Convert network_output into One-Hot vector notation
network_output = np_utils.to_categorical(network_output)

In [22]:
print(network_output.shape)
print(network_output)

(60398, 359)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Create LSTM model architecture

In [0]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [0]:
# Define model and layers

model = Sequential()

model.add (LSTM(units=512, input_shape = (network_input.shape[1], network_input.shape[2]), return_sequences = True))
model.add (Dropout(0.3))
model.add (LSTM(512, return_sequences = True))
model.add (Dropout(0.3))
model.add (LSTM(512))
model.add (Dense (256))
model.add (Dropout(0.3))
model.add (Dense (vocab_len, activation="softmax"))


In [0]:
model.compile (loss = "categorical_crossentropy", optimizer="adam")

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [0]:
checkpoint = ModelCheckpoint ("model.hdf5", monitor="loss", verbose=0, save_best_only=True, mode='min')

model_history = model.fit (network_input, network_output, epochs=100, batch_size=64, callbacks=[checkpoint])


In [0]:
model = load_model ("model.hdf5")

### Predict

In [0]:
# Create a mapping from int to element
int_to_element = dict ((num, element) for num, element in enumerate (pitch_names))

In [0]:
test_input = []

for i in range (len(notes) - sequence_length):
    seq_inp = notes[i:i+sequence_length]
    test_input.append([element_to_int[ch] for ch in seq_inp])

In [0]:
# Randomly select a sequence of notes from test_input
start = np.random.randint(len(test_input)-1)

# Feed this to model and get prediction
pattern = test_input[start]
final_prediction = []

for note_index in range(200):
    pred_inp = np.reshape(pattern, (1, len(pattern), 1))
    inp = pred_inp/float(vocab_len)

    prediction = model.predict(inp, verbose=0)
    idx = np.argmax(prediction)
    result = int_to_element[idx]

    # Append this predicted note output to final_prediction
    final_prediction.append(result)

    # Next input to model should also have same size
    # So use this same input, but drop the first note and append the predicted note
    pattern = pattern[1:]
    pattern.append(idx)
    

In [44]:
print(final_prediction)

['F2', 'B-2', 'B-2', 'C#5', 'A2', 'E-5', 'E5', 'E-5', 'C#5', 'A2', 'B4', 'A4', 'A2', 'G#4', 'B4', 'A4', 'A2', 'F#4', 'G#4', 'A2', 'A2', 'F#4', 'C5', 'F#5', 'A2', 'F#2', 'G#4', 'D5', 'A2', 'F#2', 'A2', 'F#2', 'F#4', 'F#4', 'A4', 'F#2', 'F#3', 'F#2', 'B4', 'F#2', 'F#2', 'F#2', 'F#4', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F2', 'F2', 'F#2', 'F#2', 'F2', 'F#2', 'F#2', 'C#5', 'F#2', 'C#4', 'E2', 'G#4', 'F#4', 'F4', 'E4', 'E4', 'G#4', 'F#5', 'B-4', 'F2', 'C5', 'F#5', 'F#4', 'C#4', 'D5', '1+7', 'D2', 'F3', 'G4', 'F2', 'F4', 'F2', 'F2', '8+0', 'G#2', 'E4', '5+8', 'F2', 'E-4', 'F2', 'F3', 'F2', 'F3', '5+8', 'F2', 'C4', '5+8', 'F2', 'E-4', 'F2', 'C4', 'C#3', 'F3', 'G#2', 'B-4', 'E-4', 'F4', 'G#5', 'C5', 'F#3', 'F#4', 'G#4', 'E4', 'F#4', 'B-4', 'F#2', '4+9', 'F#5', 'A4', 'C4', '3', 'E2', 'C5', '2+6', 'G2', 'C4', 'F2', 'C4', 'C3', '3+8', 'E-5', 'C4', 'C3', 'E-4', 'C4', 'C3', 'A4', 'F#3', 'F3', 'C5', 'C4', 'F#2', 'F#3', 'F#3'

### Create MIDI files

In [0]:
offset = 0  #Time
final_notes = []

for pattern in final_prediction:
    #If pattern is a chord
    if ('+' in pattern ) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        
        for curr_note in notes_in_chord:
            # For each note in the chord, create a new Note object
            new_note = note.Note(int(curr_note))
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)

        new_chord = chord.Chord(temp_notes)
        new_chord.offset = offset
        final_notes.append(new_chord)

    #If pattern is a note
    else:
        curr_note = note.Note(final_prediction[4])
        curr_note.offset = offset
        curr_note.storedInstrument = instrument.Piano()
        final_notes.append(curr_note)

    offset += 0.5 #---> A disadvantage of this model as we always set this

In [51]:
print(final_notes)

[<music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.n

In [53]:
# Create a stream object from the generated notes
MIDI_stream = stream.Stream(final_notes)
MIDI_stream.write('midi', fp = 'result_output.mid')

'result_output.mid'

In [54]:
# Listen the music
MIDI_stream.show('midi')